In [6]:
from fastfusion import Specification
spec = Specification.from_yaml(
    "architecture/four_level.arch.yaml",
    "workloads/mha_full_new.yaml"
)
spec.estimate_energy_area()
spec.component_energy.to_yaml("ERT.yaml")
spec.component_area.to_yaml("ART.yaml")
# spec.workload.mermaid_graph() # pip3 install mermaid-py
# spec.workload.einsums

2025-05-23 19:38:47 INFO        Loading yaml file architecture/four_level.arch.yaml
2025-05-23 19:38:47 INFO        Found top key variables in architecture/four_level.arch.yaml
2025-05-23 19:38:47 INFO        Found top key architecture in architecture/four_level.arch.yaml
2025-05-23 19:38:47 INFO        Found top key component_classes in architecture/four_level.arch.yaml
2025-05-23 19:38:47 INFO        Loading yaml file workloads/mha_full_new.yaml
2025-05-23 19:38:47 INFO        Found top key workload in workloads/mha_full_new.yaml
2025-05-23 19:38:47 INFO        Loading plug-ins from /home/plug-ins/neurosim-plug-in/main.py. Errors below are likely due to the plug-in.
2025-05-23 19:38:47 INFO        Added estimator NeurosimADC that estimates array_adc with actions add, compute, convert, read, shift_add, update, write, leak
2025-05-23 19:38:47 INFO        Added estimator NeurosimAdder that estimates adder with actions add, compute, convert, read, shift_add, update, write, leak
2025-05-2

In [7]:
from fastfusion.frontend.renames import Renames
from fastfusion.frontend.workload import Workload
workload = Workload.from_yaml(
    "workloads/mha_full_new.yaml",
)
renames = Renames.from_yaml(
    "workloads/mha_full_new.renames.yaml",
)
for e in workload.einsums:
    print(e)
    symbol_table = workload.get_constraint_symbol_table(e.name, renames)
    print(f'Einsum {e.name}:')
    for k, v in symbol_table.items():
        print(f'\t{k}: {v}')

2025-05-23 19:38:50 INFO        Loading yaml file workloads/mha_full_new.yaml
2025-05-23 19:38:50 INFO        Found top key workload in workloads/mha_full_new.yaml
2025-05-23 19:38:50 WARNING     Trying to parse a single element dictionary as a Workload. 
2025-05-23 19:38:50 INFO        Loading yaml file workloads/mha_full_new.renames.yaml
2025-05-23 19:38:50 INFO        Found top key renames in workloads/mha_full_new.renames.yaml
2025-05-23 19:38:50 WARNING     Trying to parse a single element dictionary as a Renames. 


name='V' tensor_accesses=[TensorAccess(name='I', projection={'B': 'b', 'M': 'm', 'D': 'd'}, output=False, factors=['b', 'm', 'd']), TensorAccess(name='WV', projection={'H': 'h', 'E': 'e', 'D': 'd'}, output=False, factors=['h', 'e', 'd']), TensorAccess(name='V', projection={'B': 'b', 'M': 'm', 'H': 'h', 'E': 'e'}, output=True, factors=['b', 'm', 'h', 'e'])] shape=[]
Einsum V:
	All: {I, V, WV}
	Inputs: {I, WV}
	Outputs: {V}
	Intermediates: {V}
	Shared: {I, V}
	WV: {WV}
	V: {V}
	I: {I}
	m: {m}
	e: {e}
	b: {b}
	h: {h}
	d: {d}
name='K' tensor_accesses=[TensorAccess(name='I', projection={'B': 'b', 'M': 'm', 'D': 'd'}, output=False, factors=['b', 'm', 'd']), TensorAccess(name='WK', projection={'H': 'h', 'E': 'e', 'D': 'd'}, output=False, factors=['h', 'e', 'd']), TensorAccess(name='K', projection={'B': 'b', 'M': 'm', 'H': 'h', 'E': 'e'}, output=True, factors=['b', 'm', 'h', 'e'])] shape=[]
Einsum K:
	All: {I, K, WK}
	Inputs: {I, WK}
	Outputs: {K}
	Intermediates: {K}
	Shared: {I, K}
	K: {K}
	W

In [8]:
# import copy
# import itertools
# from typing import Generator
# from fastfusion.frontend._set_parse import InvertibleSet
# from fastfusion.frontend.arch import Leaf, Storage
# from fastfusion.frontend.workload.workload_spec import RankVariable, Tensor


# einsum_name = "Q"
# einsum = workload.einsums[einsum_name]
# rank_variables = einsum.rank_variables
# tensors = einsum.tensors
# symbol_table = workload.get_constraint_symbol_table(einsum_name, renames)
# first_value = next(iter(symbol_table.values()))
# arch_nodes = spec.get_flattened_architecture()
# tensor2rank_variables = einsum.tensor2rank_variables

# # for node in arch_nodes:
# #     storage_constraints = node.constraints.storage._parse(symbol_table)
# #     print(f'{node.name}:')
# #     for k, v in storage_constraints.items():
# #         print(f'\t{k}: {v}')

# #     keep_choice = first_value.to_my_space(set(("Q", "K")))
# #     symbol_table[node.name] = keep_choice

# #     temporal = node.constraints.temporal._parse(symbol_table)
# #     spatial_X = node.constraints.get_spatial_constraint(for_X=True)._parse(symbol_table)
# #     spatial_Y = node.constraints.get_spatial_constraint(for_Y=True)._parse(symbol_table)

# #     for k, v in temporal.items():
# #         print(f'\tTemporal {k}: {v}')
# #     for k, v in spatial_X.items():
# #         print(f'\tSpatial X {k}: {v}')
# #     for k, v in spatial_Y.items():
# #         print(f'\tSpatial Y {k}: {v}')
        

# from itertools import chain, combinations
# def powerset(iterable):
#     "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
#     s = list(iterable)
#     return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

# def make_storage_choices_one_level(node: Leaf, symbol_table: dict[str, InvertibleSet]):
#     if not isinstance(node, Storage):
#         yield set(), symbol_table
#         return
#     new_symbol_table = copy.copy(symbol_table)
#     storage_constraints = node.constraints.storage._parse(symbol_table)
#     must_keep = first_value.to_my_space(storage_constraints["keep"])
#     must_bypass = first_value.to_my_space(storage_constraints["bypass"])
    
#     if must_keep - new_symbol_table["All"]:
#         raise KeyError(f"Keep constraint for {node.name} includes tensors that are not in the einsum: {must_keep - new_symbol_table['All']}")
#     if must_bypass - new_symbol_table["All"]:
#         raise KeyError(f"Bypass constraint for {node.name} includes tensors that are not in the einsum: {must_bypass - tensors}")
#     if must_keep & must_bypass:
#         raise KeyError(f"Keep and bypass constraints for {node.name} intersect: {must_keep & must_bypass}")
    
    
#     may_keep = tensors - must_bypass - must_keep
#     for subset in powerset(may_keep):
#         subset = first_value.to_my_space(set(subset))
#         keep_choice = first_value.to_my_space(subset | must_keep)
#         keep_choice.tensors = lambda: keep_choice # So users can do MainMemory().tensors(). Optional.
#         new_symbol_table[node.name] = keep_choice
#         assert not any(isinstance(k, str) for k in keep_choice)
#         yield keep_choice, new_symbol_table

# def make_temporal_loop_choices(node: Storage, symbol_table: dict[str, InvertibleSet]) -> Generator[list[str], None, None]:
#     if not isinstance(node, Storage):
#         yield set()
#         return
    
#     temporal_constraints = node.constraints.temporal._parse(symbol_table)
#     for i in range(len(temporal_constraints.loop_order)):
#         for j in range(i + 1, len(temporal_constraints.loop_order)):
#             if temporal_constraints.loop_order[i] & temporal_constraints.loop_order[j]:
#                 raise ValueError(
#                     f"Loop order constraint indices {i} and {j} for {node.name} intersect:"
#                     f" {temporal_constraints.loop_order[i] & temporal_constraints.loop_order[j]}"
#                 )

#     loop_order_constraint = temporal_constraints.loop_order
#     loop_order_ranks = set.union(*(set(s) for s in loop_order_constraint), set())
#     def is_valid_loop_order(loop_order):
#         loop_order = [l for l in loop_order if l in loop_order_ranks]
#         if not loop_order:
#             return True

#         i = 0
#         for rank_variable in loop_order:
#             if i >= len(loop_order_constraint):
#                 return False
#             while rank_variable not in loop_order_constraint[i]:
#                 i += 1
#                 if i >= len(loop_order_constraint):
#                     return False
#         return True
    
#     to_permute = rank_variables - temporal_constraints.loop_bounds.get_constrained_one_rank_variables()
#     for loop_order in itertools.permutations(to_permute, len(to_permute)):
#         if is_valid_loop_order(loop_order):
#             yield loop_order
            
# def make_storage_choices_all_levels(nodes: list[Storage], symbol_table: dict[str, InvertibleSet]):
#     while nodes and not isinstance(nodes[0], Storage):
#         nodes = nodes[1:]
#     if len(nodes) == 0:
#         yield dict(), symbol_table
#         return

#     for choice, symbol_table in make_storage_choices_one_level(nodes[0], symbol_table):
#         for subchoices, symbol_table in make_storage_choices_all_levels(nodes[1:], symbol_table):
#             yield {**subchoices, nodes[0].name: choice}, symbol_table

    
# # UNEVEN = True        
# # def uneven_storage(node: Leaf, symbol_table: dict[str, InvertibleSet], storage_choice: set[str], loop_order: list[str], previous_loop=None):
# #     if not UNEVEN:
# #         return {t: 0 for t in storage_choice}
    
# #     if previous_loop is None:
# #         previous_loop = set()
# #     for i, t in enumerate(loop_order):
# #         if t in previous_loop:
# #             continue
# #         previous_loop.add(t)
# #         yield from uneven_storage(node, symbol_table, storage_choice, loop_order, previous_loop)

# def place_tensor(node: Storage, tensor: Tensor, mapping: list[str]) -> Generator[list[str], None, None]:
#     if len([x for x in mapping if isinstance(x, RankVariable)]) <= 1:
#         yield [tensor] + mapping
#         return
    
#     last_mapping_appearance = max((i for i, t in enumerate(mapping) if t == tensor), default=None)
#     appeared_before = True
#     if last_mapping_appearance is None:
#         appeared_before = False
#         last_mapping_appearance = 0

#     for i in range(last_mapping_appearance, len(mapping) - 1):
#         # We're not worried about relative ordering of storage. Only storages vs. loops.
#         if not isinstance(mapping[i], RankVariable):
#             continue

#         if not appeared_before:
#             cur_irrelevant = mapping[i] not in tensor2rank_variables[tensor]
#             if cur_irrelevant:
#                 break

#         prev_loop = i - 1
#         while prev_loop > 0 and not isinstance(mapping[prev_loop], RankVariable):
#             prev_loop -= 1

#         prev_relevant = True
#         if prev_loop >= 0:
#             prev_relevant = mapping[prev_loop] in tensor2rank_variables[tensor]
#         next_irrelevant = mapping[i+1] not in tensor2rank_variables[tensor]

#         if prev_relevant and next_irrelevant:
#             mapping2 = copy.copy(mapping)
#             mapping2.insert(i, tensor)
#             yield mapping2

# def place_tensors(node: Storage, tensors: set[Tensor], mapping: list[str]):
#     tensors = list(tensors)
#     if not tensors:
#         yield mapping
#         return
    
#     for mapping2 in place_tensor(node, tensors[0], mapping):
#         yield from place_tensors(node, tensors[1:], mapping2)
        
# def make_temporal_choices_and_place_tensors(node: Storage, tensors: set[Tensor], symbol_table: dict[str, InvertibleSet], mapping: list[str]=None):
#     mapping = [] if mapping is None else mapping
#     for mapping2 in place_tensors(node, tensors, mapping):
#         for loop_order in make_temporal_loop_choices(node, symbol_table):
#             yield mapping2 + list(loop_order) 
            
            
# # seen = set()
# def check_equivalency(mapping: list[str], remaining_nodes: int, tensors_left_to_place: set[Tensor]) -> bool:
#     if remaining_nodes == 0:
#         check_up_to = len(mapping)
#     elif not all(t in mapping for t in tensors_left_to_place):
#         return mapping
#     else:
#         mapping.reverse()
#         check_up_to = max(len(mapping) - mapping.index(t) - 1 for t in tensors_left_to_place)
#         mapping.reverse()
    
#     # Sort loop order between any pair of tensors
#     for i in range(check_up_to):
#         if not isinstance(mapping[i], RankVariable):
#             continue
#         if i > 0 and isinstance(mapping[i-1], RankVariable):
#             continue
#         if i < len(mapping) - 1 and not isinstance(mapping[i+1], RankVariable):
#             continue
        
#         j = i + 1
#         while j < len(mapping) - 1 and isinstance(mapping[j+1], RankVariable):
#             j += 1
        
#         mapping[i:j+1] = sorted(mapping[i:j+1])
        
#     # Drop duplicate rank variables
#     for i in range(check_up_to - 1, 0, -1):
#         if not isinstance(mapping[i], RankVariable):
#             continue
#         if mapping[i] == mapping[i-1]:
#             mapping.pop(i)
            
#     key = (remaining_nodes, tuple(mapping))
#     if key in seen:
#         return None
#     seen.add(key)
#     return mapping
            
            
# def make_temporal_choices_and_place_tensors_all_levels(nodes: list[Storage], storage_choices: dict[str, set[Tensor]], symbol_table: dict[str, InvertibleSet], mapping: list[str]=None):
#     while nodes and not isinstance(nodes[0], Storage):
#         nodes = nodes[1:]

#     mapping = [] if mapping is None else mapping
    
#     tensors_left_to_place = set().union(*(storage_choices.get(n.name, set()) for n in nodes))

#     if len(nodes) == 0:
#         mapping = check_equivalency(mapping, len(nodes), tensors_left_to_place)
#         if mapping is not None:
#             yield mapping
#         return
    
#     if len(mapping) > 30:
#         print("AHH")
    
#     tensors = storage_choices[nodes[0].name]
    
#     for mapping2 in make_temporal_choices_and_place_tensors(nodes[0], tensors, symbol_table, mapping):
#         mapping2 = check_equivalency(mapping2, len(nodes), tensors_left_to_place)
#         if mapping2 is None:
#             continue
#         for mapping3 in make_temporal_choices_and_place_tensors_all_levels(nodes[1:], storage_choices, symbol_table, mapping2):
#             yield mapping3

# # def place_storage(node: Storage, storage_choices: set[str], mapping: list[str]):
# #     mapping = copy.deepcopy(mapping)
# #     # storage_options = {}

# #     # for tensor in storage_choices.get(node.name, []):
# #     #     last_mapping_appearance = max((i for i, t in enumerate(mapping) if t == tensor), default=0)
# #     #     for j in range(last_mapping_appearance, len(mapping)):
# #     #         prev_loop = mapping[j-1] if j > 0 else None
# #     #         prev_relevant = j == 0 or mapping[j-1] in tensor2rank_variables[tensor]
            
# # TODO: Move parsing to after generating all storage choices
        
# arch_nodes = [n for n in arch_nodes if isinstance(n, Storage)]
    
# i = 0
# for storage_choices, symbol_table2 in make_storage_choices_all_levels(arch_nodes, symbol_table):
#     seen = set()
#     print(f'\n\nStorage choices: {storage_choices}')
#     for mapping in make_temporal_choices_and_place_tensors_all_levels(arch_nodes, storage_choices, symbol_table2):
#         # print(f'\t{mapping}')
#         i += 1
#         # if i % 1000 == 0:
#         print(f'\t{i}: {mapping}')

In [9]:
# import copy
# import itertools
# from typing import Generator
# from fastfusion.frontend._set_parse import InvertibleSet
# from fastfusion.frontend.arch import Leaf, Storage
# from fastfusion.frontend.workload.workload_spec import RankVariable, Tensor


# einsum_name = "Q"
# einsum = workload.einsums[einsum_name]
# rank_variables = einsum.rank_variables
# tensors = einsum.tensors
# symbol_table = workload.get_constraint_symbol_table(einsum_name, renames)
# first_value = next(iter(symbol_table.values()))
# arch_nodes = spec.get_flattened_architecture()
# tensor2rank_variables = einsum.tensor2rank_variables

# from itertools import chain, combinations
# def powerset(iterable):
#     "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
#     s = list(iterable)
#     return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

# def make_storage_choices_one_level(node: Leaf, symbol_table: dict[str, InvertibleSet]):
#     if not isinstance(node, Storage):
#         yield set(), symbol_table
#         return
#     new_symbol_table = copy.copy(symbol_table)
#     storage_constraints = node.constraints.storage._parse(symbol_table)
#     must_keep = first_value.to_my_space(storage_constraints["keep"])
#     must_bypass = first_value.to_my_space(storage_constraints["bypass"])
    
#     if must_keep - new_symbol_table["All"]:
#         raise KeyError(f"Keep constraint for {node.name} includes tensors that are not in the einsum: {must_keep - new_symbol_table['All']}")
#     if must_bypass - new_symbol_table["All"]:
#         raise KeyError(f"Bypass constraint for {node.name} includes tensors that are not in the einsum: {must_bypass - tensors}")
#     if must_keep & must_bypass:
#         raise KeyError(f"Keep and bypass constraints for {node.name} intersect: {must_keep & must_bypass}")
    
    
#     may_keep = tensors - must_bypass - must_keep
#     for subset in powerset(may_keep):
#         subset = first_value.to_my_space(set(subset))
#         keep_choice = first_value.to_my_space(subset | must_keep)
#         keep_choice.tensors = lambda: keep_choice # So users can do MainMemory().tensors(). Optional.
#         new_symbol_table[node.name] = keep_choice
#         assert not any(isinstance(k, str) for k in keep_choice)
#         yield keep_choice, new_symbol_table

# def make_temporal_loop_choices(node: Storage, symbol_table: dict[str, InvertibleSet]) -> Generator[list[str], None, None]:
#     if not isinstance(node, Storage):
#         yield set()
#         return
    
#     temporal_constraints = node.constraints.temporal._parse(symbol_table)
#     for i in range(len(temporal_constraints.loop_order)):
#         for j in range(i + 1, len(temporal_constraints.loop_order)):
#             if temporal_constraints.loop_order[i] & temporal_constraints.loop_order[j]:
#                 raise ValueError(
#                     f"Loop order constraint indices {i} and {j} for {node.name} intersect:"
#                     f" {temporal_constraints.loop_order[i] & temporal_constraints.loop_order[j]}"
#                 )

#     loop_order_constraint = temporal_constraints.loop_order
#     loop_order_ranks = set.union(*(set(s) for s in loop_order_constraint), set())
#     def is_valid_loop_order(loop_order):
#         loop_order = [l for l in loop_order if l in loop_order_ranks]
#         if not loop_order:
#             return True

#         i = 0
#         for rank_variable in loop_order:
#             if i >= len(loop_order_constraint):
#                 return False
#             while rank_variable not in loop_order_constraint[i]:
#                 i += 1
#                 if i >= len(loop_order_constraint):
#                     return False
#         return True
    
#     to_permute = rank_variables - temporal_constraints.loop_bounds.get_constrained_one_rank_variables()
#     for loop_order in itertools.permutations(to_permute, len(to_permute)):
#         if is_valid_loop_order(loop_order):
#             yield loop_order
            
# def make_storage_choices_all_levels(nodes: list[Storage], symbol_table: dict[str, InvertibleSet]):
#     while nodes and not isinstance(nodes[0], Storage):
#         nodes = nodes[1:]
#     if len(nodes) == 0:
#         yield dict(), symbol_table
#         return

#     for choice, symbol_table in make_storage_choices_one_level(nodes[0], symbol_table):
#         for subchoices, symbol_table in make_storage_choices_all_levels(nodes[1:], symbol_table):
#             yield {**subchoices, nodes[0].name: choice}, symbol_table

    
# # UNEVEN = True        
# # def uneven_storage(node: Leaf, symbol_table: dict[str, InvertibleSet], storage_choice: set[str], loop_order: list[str], previous_loop=None):
# #     if not UNEVEN:
# #         return {t: 0 for t in storage_choice}
    
# #     if previous_loop is None:
# #         previous_loop = set()
# #     for i, t in enumerate(loop_order):
# #         if t in previous_loop:
# #             continue
# #         previous_loop.add(t)
# #         yield from uneven_storage(node, symbol_table, storage_choice, loop_order, previous_loop)

# def place_tensor(node: Storage, tensor: Tensor, mapping: list[str]) -> Generator[list[str], None, None]:
#     if len([x for x in mapping if isinstance(x, RankVariable)]) <= 1:
#         yield [tensor] + mapping
#         return
    
#     last_mapping_appearance = max((i for i, t in enumerate(mapping) if t == tensor), default=None)
#     appeared_before = True
#     if last_mapping_appearance is None:
#         appeared_before = False
#         last_mapping_appearance = 0

#     for i in range(last_mapping_appearance, len(mapping) - 1):
#         # We're not worried about relative ordering of storage. Only storages vs. loops.
#         if not isinstance(mapping[i], RankVariable):
#             continue

#         if not appeared_before:
#             cur_irrelevant = mapping[i] not in tensor2rank_variables[tensor]
#             if cur_irrelevant:
#                 break

#         prev_loop = i - 1
#         while prev_loop > 0 and not isinstance(mapping[prev_loop], RankVariable):
#             prev_loop -= 1

#         prev_relevant = True
#         if prev_loop >= 0:
#             prev_relevant = mapping[prev_loop] in tensor2rank_variables[tensor]
#         next_irrelevant = mapping[i+1] not in tensor2rank_variables[tensor]

#         if prev_relevant and next_irrelevant:
#             mapping2 = copy.copy(mapping)
#             mapping2.insert(i, tensor)
#             yield mapping2

# def place_tensors(node: Storage, tensors: set[Tensor], mapping: list[str]):
#     tensors = list(tensors)
#     if not tensors:
#         yield mapping
#         return
    
#     for mapping2 in place_tensor(node, tensors[0], mapping):
#         yield from place_tensors(node, tensors[1:], mapping2)
        
# def make_temporal_choices_and_place_tensors(node: Storage, tensors: set[Tensor], symbol_table: dict[str, InvertibleSet], mapping: list[str]=None):
#     mapping = [] if mapping is None else mapping
#     for mapping2 in place_tensors(node, tensors, mapping):
#         for loop_order in make_temporal_loop_choices(node, symbol_table):
#             yield mapping2 + list(loop_order) 
            
            
# # seen = set()
# def check_equivalency(mapping: list[str], remaining_nodes: int, tensors_left_to_place: set[Tensor]) -> bool:
#     if remaining_nodes == 0:
#         check_up_to = len(mapping)
#     elif not all(t in mapping for t in tensors_left_to_place):
#         return mapping
#     else:
#         mapping.reverse()
#         check_up_to = max(len(mapping) - mapping.index(t) - 1 for t in tensors_left_to_place)
#         mapping.reverse()
    
#     # Sort loop order between any pair of tensors
#     for i in range(check_up_to):
#         if not isinstance(mapping[i], RankVariable):
#             continue
#         if i > 0 and isinstance(mapping[i-1], RankVariable):
#             continue
#         if i < len(mapping) - 1 and not isinstance(mapping[i+1], RankVariable):
#             continue
        
#         j = i + 1
#         while j < len(mapping) - 1 and isinstance(mapping[j+1], RankVariable):
#             j += 1
        
#         mapping[i:j+1] = sorted(mapping[i:j+1])
        
#     # Drop duplicate rank variables
#     for i in range(check_up_to - 1, 0, -1):
#         if not isinstance(mapping[i], RankVariable):
#             continue
#         if mapping[i] == mapping[i-1]:
#             mapping.pop(i)
            
#     key = (remaining_nodes, tuple(mapping))
#     if key in seen:
#         return None
#     seen.add(key)
#     return mapping
            
            
# def make_temporal_choices_and_place_tensors_all_levels(nodes: list[Storage], storage_choices: dict[str, set[Tensor]], symbol_table: dict[str, InvertibleSet], mapping: list[str]=None):
#     while nodes and not isinstance(nodes[0], Storage):
#         nodes = nodes[1:]

#     mapping = [] if mapping is None else mapping
    
#     tensors_left_to_place = set().union(*(storage_choices.get(n.name, set()) for n in nodes))

#     if len(nodes) == 0:
#         mapping = check_equivalency(mapping, len(nodes), tensors_left_to_place)
#         if mapping is not None:
#             yield mapping
#         return
    
#     if len(mapping) > 30:
#         print("AHH")
    
#     tensors = storage_choices[nodes[0].name]
    
#     for mapping2 in make_temporal_choices_and_place_tensors(nodes[0], tensors, symbol_table, mapping):
#         mapping2 = check_equivalency(mapping2, len(nodes), tensors_left_to_place)
#         if mapping2 is None:
#             continue
#         for mapping3 in make_temporal_choices_and_place_tensors_all_levels(nodes[1:], storage_choices, symbol_table, mapping2):
#             yield mapping3

# # def place_storage(node: Storage, storage_choices: set[str], mapping: list[str]):
# #     mapping = copy.deepcopy(mapping)
# #     # storage_options = {}

# #     # for tensor in storage_choices.get(node.name, []):
# #     #     last_mapping_appearance = max((i for i, t in enumerate(mapping) if t == tensor), default=0)
# #     #     for j in range(last_mapping_appearance, len(mapping)):
# #     #         prev_loop = mapping[j-1] if j > 0 else None
# #     #         prev_relevant = j == 0 or mapping[j-1] in tensor2rank_variables[tensor]
            
# # TODO: Move parsing to after generating all storage choices
        
# # arch_nodes = [n for n in arch_nodes if isinstance(n, Storage)]
    
# # i = 0
# # for storage_choices, symbol_table2 in make_storage_choices_all_levels(arch_nodes, symbol_table):
# #     seen = set()
# #     print(f'\n\nStorage choices: {storage_choices}')
# #     for mapping in make_temporal_choices_and_place_tensors_all_levels(arch_nodes, storage_choices, symbol_table2):
# #         # print(f'\t{mapping}')
# #         i += 1
# #         # if i % 1000 == 0:
# #         print(f'\t{i}: {mapping}')
        
# def get_storage_loop_blocks(i: int, mapping: list[str]) -> tuple[list[Tensor], list[RankVariable], list[Tensor]]:
#     if not isinstance(mapping[i], RankVariable):
#         raise ValueError(f"Index {i} must point to a RankVariable")
        
#     left_tensors = []
#     right_tensors = []
    
#     # Find the start of the rank variable block
#     rank_start = i
#     while rank_start > 0 and isinstance(mapping[rank_start - 1], RankVariable):
#         rank_start -= 1
    
#     # Find the end of the rank variable block
#     rank_end = i
#     while rank_end < len(mapping) - 1 and isinstance(mapping[rank_end + 1], RankVariable):
#         rank_end += 1
    
#     # Find the start of the left tensor block
#     left_tensor_start = rank_start
#     while left_tensor_start > 0 and isinstance(mapping[left_tensor_start - 1], Tensor):
#         left_tensor_start -= 1
    
#     # Find the end of the right tensor block
#     right_tensor_end = rank_end
#     while right_tensor_end < len(mapping) - 1 and isinstance(mapping[right_tensor_end + 1], Tensor):
#         right_tensor_end += 1
    
#     # Extract the blocks
#     left_tensors = mapping[left_tensor_start:rank_start]
#     ranks = mapping[rank_start:rank_end + 1]
#     right_tensors = mapping[rank_end + 1:right_tensor_end + 1]
    
#     return left_tensors, ranks, right_tensors        
        
# def is_valid(mapping):
#     # Back-to-back rank variables must be ordered
#     for i in range(len(mapping) - 1):
#         if isinstance(mapping[i], RankVariable) and isinstance(mapping[i+1], RankVariable):
#             if mapping[i] >= mapping[i+1]:
#                 return False

#     # Back to back storages must be ordered
#     for i in range(len(mapping) - 1):
#         if isinstance(mapping[i], Storage) and isinstance(mapping[i+1], Storage):
#             if mapping[i] > mapping[i+1]:
#                 return False

#     # Above the first appearance of a tensor, all rank variables must be irrelevant
#     for t in tensors:
#         for i in range(len(mapping)):
#             if mapping[i] == t:
#                 break
#             if isinstance(mapping[i], RankVariable):
#                 if mapping[i] not in tensor2rank_variables[t]:
#                     return False
    
#     # Looking at any contiguous block of rank variables:
#     # - Grab the left contiguous block of tensors.
#     # - Grab the right contiguous block of tensors.
#     # - If the rank variable is relevant to all left tensors, it's invalid.
#     # - If the rank variable is irrelevant to all right tensors, it's invalid.
#     for i in range(len(mapping)):
#         if isinstance(mapping[i], RankVariable):
#             left_tensors, ranks, right_tensors = get_storage_loop_blocks(i, mapping)
#             if left_tensors:
#                 if any(all(r in tensor2rank_variables[t] for t in left_tensors) for r in ranks):
#                     return False
#             if right_tensors:
#                 if any(all(r not in tensor2rank_variables[t] for t in right_tensors) for r in ranks):
#                     return False
#     return True
    
# def recursive_build_mapping(
#     mapping: list[str],
#     nodes: list[Storage],
#     tensor2remaining: dict[Tensor, int],
# ):
#     if not tensor2remaining or not any(tensor2remaining.values()):
#         yield mapping
#         return
    
#     choices = [t for t in tensor2remaining if tensor2remaining[t] > 0]
#     choices += rank_variables
    
#     for choice in choices:
#         mapping2 = mapping + [choice]
#         # Check for violation
#         if is_valid(mapping2):
#             if choice in tensors:
#                 tensor2remaining[choice] -= 1
#             yield from recursive_build_mapping(mapping2, nodes, tensor2remaining)
#             if choice in tensors:
#                 tensor2remaining[choice] += 1
    
    
# # If there are two back-to-back storages for the same tensor & the outer is
# # optional, then it is invalid.

# storage_choice_options = [s for s, _ in make_storage_choices_all_levels(arch_nodes, symbol_table)]

# for storage_choices in storage_choice_options[8:]:
#     print(f'{storage_choices}')
#     main_memory = arch_nodes[0]
#     tensor2remaining = {t: 0 for t in tensors}
#     for k, v in storage_choices.items():
#         if k != main_memory.name:
#             for t in v:
#                 tensor2remaining[t] += 1
#     mapping_base = list(storage_choices[main_memory.name])
#     for mapping in recursive_build_mapping(mapping_base, arch_nodes[1:], tensor2remaining):
#         print(mapping)
#         pass

In [10]:
import copy
import itertools
from typing import Generator
from fastfusion.frontend._set_parse import InvertibleSet
from fastfusion.frontend.architecture import Leaf, Storage
from fastfusion.frontend.workload.workload import RankVariable, Tensor


einsum_name = "Q"
einsum = workload.einsums[einsum_name]
rank_variables = einsum.rank_variables
tensors = einsum.tensors
symbol_table = workload.get_constraint_symbol_table(einsum_name, renames)
first_value = next(iter(symbol_table.values()))
arch_nodes = spec.get_flattened_architecture()
tensor2rank_variables = einsum.tensor2rank_variables

from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

def make_storage_choices_one_level(node: Leaf, symbol_table: dict[str, InvertibleSet]):
    if not isinstance(node, Storage):
        yield set(), symbol_table
        return
    new_symbol_table = copy.copy(symbol_table)
    storage_constraints = node.constraints.storage._parse(symbol_table)
    must_keep = first_value.to_my_space(storage_constraints["keep"])
    must_bypass = first_value.to_my_space(storage_constraints["bypass"])
    
    if must_keep - new_symbol_table["All"]:
        raise KeyError(f"Keep constraint for {node.name} includes tensors that are not in the einsum: {must_keep - new_symbol_table['All']}")
    if must_bypass - new_symbol_table["All"]:
        raise KeyError(f"Bypass constraint for {node.name} includes tensors that are not in the einsum: {must_bypass - tensors}")
    if must_keep & must_bypass:
        raise KeyError(f"Keep and bypass constraints for {node.name} intersect: {must_keep & must_bypass}")
    
    
    may_keep = tensors - must_bypass - must_keep
    for subset in powerset(may_keep):
        subset = first_value.to_my_space(set(subset))
        keep_choice = first_value.to_my_space(subset | must_keep)
        keep_choice.tensors = lambda: keep_choice # So users can do MainMemory().tensors(). Optional.
        new_symbol_table[node.name] = keep_choice
        assert not any(isinstance(k, str) for k in keep_choice)
        yield keep_choice, new_symbol_table

def make_temporal_loop_choices(node: Storage, symbol_table: dict[str, InvertibleSet]) -> Generator[list[str], None, None]:
    if not isinstance(node, Storage):
        yield set()
        return
    
    temporal_constraints = node.constraints.temporal._parse(symbol_table)
    for i in range(len(temporal_constraints.loop_order)):
        for j in range(i + 1, len(temporal_constraints.loop_order)):
            if temporal_constraints.loop_order[i] & temporal_constraints.loop_order[j]:
                raise ValueError(
                    f"Loop order constraint indices {i} and {j} for {node.name} intersect:"
                    f" {temporal_constraints.loop_order[i] & temporal_constraints.loop_order[j]}"
                )

    loop_order_constraint = temporal_constraints.loop_order
    loop_order_ranks = set.union(*(set(s) for s in loop_order_constraint), set())
    def is_valid_loop_order(loop_order):
        loop_order = [l for l in loop_order if l in loop_order_ranks]
        if not loop_order:
            return True

        i = 0
        for rank_variable in loop_order:
            if i >= len(loop_order_constraint):
                return False
            while rank_variable not in loop_order_constraint[i]:
                i += 1
                if i >= len(loop_order_constraint):
                    return False
        return True
    
    to_permute = rank_variables - temporal_constraints.loop_bounds.get_constrained_one_rank_variables()
    for loop_order in itertools.permutations(to_permute, len(to_permute)):
        if is_valid_loop_order(loop_order):
            yield loop_order
            
def make_storage_choices_all_levels(nodes: list[Storage], symbol_table: dict[str, InvertibleSet]):
    while nodes and not isinstance(nodes[0], Storage):
        nodes = nodes[1:]
    if len(nodes) == 0:
        yield dict(), symbol_table
        return

    for choice, symbol_table in make_storage_choices_one_level(nodes[0], symbol_table):
        for subchoices, symbol_table in make_storage_choices_all_levels(nodes[1:], symbol_table):
            yield {**subchoices, nodes[0].name: choice}, symbol_table

    
# UNEVEN = True        
# def uneven_storage(node: Leaf, symbol_table: dict[str, InvertibleSet], storage_choice: set[str], loop_order: list[str], previous_loop=None):
#     if not UNEVEN:
#         return {t: 0 for t in storage_choice}
    
#     if previous_loop is None:
#         previous_loop = set()
#     for i, t in enumerate(loop_order):
#         if t in previous_loop:
#             continue
#         previous_loop.add(t)
#         yield from uneven_storage(node, symbol_table, storage_choice, loop_order, previous_loop)

def place_tensor(node: Storage, tensor: Tensor, mapping: list[str]) -> Generator[list[str], None, None]:
    if len([x for x in mapping if isinstance(x, RankVariable)]) <= 1:
        yield [tensor] + mapping
        return
    
    last_mapping_appearance = max((i for i, t in enumerate(mapping) if t == tensor), default=None)
    appeared_before = True
    if last_mapping_appearance is None:
        appeared_before = False
        last_mapping_appearance = 0

    for i in range(last_mapping_appearance, len(mapping) - 1):
        # We're not worried about relative ordering of storage. Only storages vs. loops.
        if not isinstance(mapping[i], RankVariable):
            continue

        if not appeared_before:
            cur_irrelevant = mapping[i] not in tensor2rank_variables[tensor]
            if cur_irrelevant:
                break

        prev_loop = i - 1
        while prev_loop > 0 and not isinstance(mapping[prev_loop], RankVariable):
            prev_loop -= 1

        prev_relevant = True
        if prev_loop >= 0:
            prev_relevant = mapping[prev_loop] in tensor2rank_variables[tensor]
        next_irrelevant = mapping[i+1] not in tensor2rank_variables[tensor]

        if prev_relevant and next_irrelevant:
            mapping2 = copy.copy(mapping)
            mapping2.insert(i, tensor)
            yield mapping2

def place_tensors(node: Storage, tensors: set[Tensor], mapping: list[str]):
    tensors = list(tensors)
    if not tensors:
        yield mapping
        return
    
    for mapping2 in place_tensor(node, tensors[0], mapping):
        yield from place_tensors(node, tensors[1:], mapping2)
        
def make_temporal_choices_and_place_tensors(node: Storage, tensors: set[Tensor], symbol_table: dict[str, InvertibleSet], mapping: list[str]=None):
    mapping = [] if mapping is None else mapping
    for mapping2 in place_tensors(node, tensors, mapping):
        for loop_order in make_temporal_loop_choices(node, symbol_table):
            yield mapping2 + list(loop_order) 
            
            
# seen = set()
def check_equivalency(mapping: list[str], remaining_nodes: int, tensors_left_to_place: set[Tensor]) -> bool:
    if remaining_nodes == 0:
        check_up_to = len(mapping)
    elif not all(t in mapping for t in tensors_left_to_place):
        return mapping
    else:
        mapping.reverse()
        check_up_to = max(len(mapping) - mapping.index(t) - 1 for t in tensors_left_to_place)
        mapping.reverse()
    
    # Sort loop order between any pair of tensors
    for i in range(check_up_to):
        if not isinstance(mapping[i], RankVariable):
            continue
        if i > 0 and isinstance(mapping[i-1], RankVariable):
            continue
        if i < len(mapping) - 1 and not isinstance(mapping[i+1], RankVariable):
            continue
        
        j = i + 1
        while j < len(mapping) - 1 and isinstance(mapping[j+1], RankVariable):
            j += 1
        
        mapping[i:j+1] = sorted(mapping[i:j+1])
        
    # Drop duplicate rank variables
    for i in range(check_up_to - 1, 0, -1):
        if not isinstance(mapping[i], RankVariable):
            continue
        if mapping[i] == mapping[i-1]:
            mapping.pop(i)
            
    key = (remaining_nodes, tuple(mapping))
    if key in seen:
        return None
    seen.add(key)
    return mapping
            
            
def make_temporal_choices_and_place_tensors_all_levels(nodes: list[Storage], storage_choices: dict[str, set[Tensor]], symbol_table: dict[str, InvertibleSet], mapping: list[str]=None):
    while nodes and not isinstance(nodes[0], Storage):
        nodes = nodes[1:]

    mapping = [] if mapping is None else mapping
    
    tensors_left_to_place = set().union(*(storage_choices.get(n.name, set()) for n in nodes))

    if len(nodes) == 0:
        mapping = check_equivalency(mapping, len(nodes), tensors_left_to_place)
        if mapping is not None:
            yield mapping
        return
    
    if len(mapping) > 30:
        print("AHH")
    
    tensors = storage_choices[nodes[0].name]
    
    for mapping2 in make_temporal_choices_and_place_tensors(nodes[0], tensors, symbol_table, mapping):
        mapping2 = check_equivalency(mapping2, len(nodes), tensors_left_to_place)
        if mapping2 is None:
            continue
        for mapping3 in make_temporal_choices_and_place_tensors_all_levels(nodes[1:], storage_choices, symbol_table, mapping2):
            yield mapping3

# def place_storage(node: Storage, storage_choices: set[str], mapping: list[str]):
#     mapping = copy.deepcopy(mapping)
#     # storage_options = {}

#     # for tensor in storage_choices.get(node.name, []):
#     #     last_mapping_appearance = max((i for i, t in enumerate(mapping) if t == tensor), default=0)
#     #     for j in range(last_mapping_appearance, len(mapping)):
#     #         prev_loop = mapping[j-1] if j > 0 else None
#     #         prev_relevant = j == 0 or mapping[j-1] in tensor2rank_variables[tensor]
            
# TODO: Move parsing to after generating all storage choices
        
# arch_nodes = [n for n in arch_nodes if isinstance(n, Storage)]
    
# i = 0
# for storage_choices, symbol_table2 in make_storage_choices_all_levels(arch_nodes, symbol_table):
#     seen = set()
#     print(f'\n\nStorage choices: {storage_choices}')
#     for mapping in make_temporal_choices_and_place_tensors_all_levels(arch_nodes, storage_choices, symbol_table2):
#         # print(f'\t{mapping}')
#         i += 1
#         # if i % 1000 == 0:
#         print(f'\t{i}: {mapping}')
        
def get_storage_loop_blocks(i: int, mapping: list[str], first_non_fused_loop: int) -> tuple[list[Tensor], list[RankVariable], list[Tensor]]:
    if not isinstance(mapping[i], RankVariable):
        raise ValueError(f"Index {i} must point to a RankVariable")
        
    left_tensors = []
    right_tensors = []
    
    # Find the start of the rank variable block
    rank_start = i
    while rank_start > 0 and isinstance(mapping[rank_start - 1], RankVariable):
        rank_start -= 1
    
    # Find the end of the rank variable block
    rank_end = i
    while rank_end < len(mapping) - 1 and isinstance(mapping[rank_end + 1], RankVariable):
        rank_end += 1
    
    # Find the start of the left tensor block
    left_tensor_start = rank_start
    while left_tensor_start > 0 and isinstance(mapping[left_tensor_start - 1], Tensor):
        left_tensor_start -= 1
    
    # Find the end of the right tensor block
    right_tensor_end = rank_end
    while right_tensor_end < len(mapping) - 1 and isinstance(mapping[right_tensor_end + 1], Tensor):
        right_tensor_end += 1
    
    # Extract the blocks
    rank_start = max(rank_start, first_non_fused_loop)
    left_tensors = mapping[left_tensor_start:rank_start]
    ranks = mapping[rank_start:rank_end + 1]
    right_tensors = mapping[rank_end + 1:right_tensor_end + 1]
    
    return left_tensors, ranks, right_tensors        
        
def is_valid(mapping):
    return True
    # Above the first appearance of a tensor, all rank variables must be irrelevant
    for t in tensors:
        for i in range(0, len(mapping)):
            if mapping[i] == t:
                break
            if isinstance(mapping[i], RankVariable):
                if mapping[i] not in tensor2rank_variables[t]:
                    return False

    # No multiple loops over the same rank
    for i in range(len(mapping)):
        for j in range(i-1, -1, -1):
            if not isinstance(mapping[j], RankVariable):
                break
            if mapping[i] == mapping[j]:
                return False

    # No further rules until all backing storages have been seen
    seen_tensors = set()
    first_non_fused_loop = len(mapping)
    for i in range(len(mapping)):
        if isinstance(mapping[i], Tensor):
            seen_tensors.add(mapping[i])
        if len(seen_tensors) == len(tensors):
            first_non_fused_loop = i
            break
        
    
    # Back-to-back rank variables must be ordered
    for i in range(first_non_fused_loop, len(mapping) - 1):
        if isinstance(mapping[i], RankVariable) and isinstance(mapping[i+1], RankVariable):
            if mapping[i] >= mapping[i+1]:
                return False

    # Back to back storages must be ordered
    for i in range(first_non_fused_loop, len(mapping) - 1):
        if isinstance(mapping[i], Storage) and isinstance(mapping[i+1], Storage):
            if mapping[i] > mapping[i+1]:
                return False
    
    # Looking at any contiguous block of rank variables:
    # - Grab the left contiguous block of tensors.
    # - Grab the right contiguous block of tensors.
    # - If the rank variable is relevant to all left tensors, it's invalid.
    # - If the rank variable is irrelevant to all right tensors, it's invalid.
    for i in range(len(mapping)):
        if not isinstance(mapping[i], RankVariable):
            continue
        left_tensors, ranks, right_tensors = get_storage_loop_blocks(i, mapping, 0)
        if left_tensors:
            if any(all(r in tensor2rank_variables[t] for t in left_tensors) for r in ranks):
                return False
        if right_tensors:
            if any(all(r not in tensor2rank_variables[t] for t in right_tensors) for r in ranks):
                return False

    # LRP: Each tensor must be immediately above an irrelevant rank variable and
    # immediately below a relevant rank variable. Else it'd be better to move
    # the tensor down or up.
    for i in range(first_non_fused_loop, len(mapping)):
        if not isinstance(mapping[i], Tensor):
            continue

        j = i - 1
        while j >= first_non_fused_loop:
            if isinstance(mapping[j], RankVariable):
                if mapping[j] not in tensor2rank_variables[mapping[i]]:
                    return False
                break
            j -= 1
        j = i + 1
        while j < len(mapping):
            if isinstance(mapping[j], RankVariable):
                if mapping[j] in tensor2rank_variables[mapping[i]]:
                    return False
                break
            j += 1
                
    return True
    
def recursive_build_mapping(
    mapping: list[str],
    nodes: list[Storage],
    tensor2remaining: dict[Tensor, int],
):
    if not tensor2remaining or not any(tensor2remaining.values()):
        yield mapping
        return
    
    choices = [t for t in tensor2remaining if tensor2remaining[t] > 0]
    choices += rank_variables
    
    for choice in choices:
        mapping2 = mapping + [choice]
        # Check for violation
        if is_valid(mapping2):
            if choice in tensors:
                tensor2remaining[choice] -= 1
            yield from recursive_build_mapping(mapping2, nodes, tensor2remaining)
            if choice in tensors:
                tensor2remaining[choice] += 1
    
    
# If there are two back-to-back storages for the same tensor & the outer is
# optional, then it is invalid.
uneven_storages = [n for n in arch_nodes if n.constraints.storage.uneven]
storage_choice_options = [s for s, _ in make_storage_choices_all_levels(arch_nodes, symbol_table)]

groups = []
current_group = []
for node in arch_nodes:
    if isinstance(node, Storage):
        if current_group and not node.constraints.storage.uneven:
            groups.append(current_group)
            current_group = []
        current_group.append(node)
if current_group:
    groups.append(current_group)


# main_memory = arch_nodes[0]
# for storage_choices in storage_choice_options[8:]:
#     print(f'{storage_choices}')

#     mapping_base = list(storage_choices[main_memory.name])
#     for group in groups:
#         print(f'\tGroup: {[n.name for n in group]}')
#         group_names = [n.name for n in group]
#         tensor2remaining = {t: 0 for t in tensors}
#         for k, v in storage_choices.items():
#             if k != main_memory.name:
#                 for t in v:
#                     tensor2remaining[t] += 1
#         for mapping in recursive_build_mapping(mapping_base, group, tensor2remaining):
#             print(f'\t\t{mapping}')
    
    
def recursive_build_from_groups(groups, storage_choices, mapping_base=None):
    mapping_base = [] if mapping_base is None else mapping_base
    mapping_base = [x for x in mapping_base]
    if not groups:
        yield mapping_base
        return
    
    group = groups[0]
    tensor2remaining = {t: 0 for t in tensors}
    for node in group:
        for t in storage_choices[node.name]:
            tensor2remaining[t] += 1

    main_memory = group[0]
    for t in storage_choices[main_memory.name]:
        tensor2remaining[t] -= 1
        mapping_base.append(t)
            
    for mapping in recursive_build_mapping(mapping_base, group, tensor2remaining):
        yield from recursive_build_from_groups(groups[1:], storage_choices, mapping)
    
import time

for storage_choices in storage_choice_options[:1]:
    print(f'{storage_choices}')
    main_memory = arch_nodes[0]

    start_time = time.time()
    mappings_count = 0
    for mapping in recursive_build_from_groups(groups, storage_choices):
        mappings_count += 1
        print(mapping)

        elapsed_time = time.time() - start_time
        mappings_per_second = mappings_count / elapsed_time if elapsed_time > 0 else 0
        # print(f"Generated {mappings_count} mappings in {elapsed_time:.2f} seconds ({mappings_per_second:.2f} mappings/second)")

ModuleNotFoundError: No module named 'fastfusion.frontend._set_parse'